In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [3]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.949, val acc: 0.114 (best 0.114), test acc: 0.103 (best 0.103)
In epoch 5, loss: 1.871, val acc: 0.498 (best 0.498), test acc: 0.514 (best 0.514)
In epoch 10, loss: 1.734, val acc: 0.364 (best 0.554), test acc: 0.392 (best 0.579)
In epoch 15, loss: 1.531, val acc: 0.358 (best 0.554), test acc: 0.363 (best 0.579)
In epoch 20, loss: 1.268, val acc: 0.476 (best 0.554), test acc: 0.472 (best 0.579)
In epoch 25, loss: 0.970, val acc: 0.602 (best 0.602), test acc: 0.609 (best 0.609)
In epoch 30, loss: 0.679, val acc: 0.686 (best 0.686), test acc: 0.683 (best 0.683)
In epoch 35, loss: 0.438, val acc: 0.712 (best 0.712), test acc: 0.718 (best 0.718)
In epoch 40, loss: 0.267, val acc: 0.724 (best 0.724), test acc: 0.723 (best 0.723)
In epoch 45, loss: 0.161, val acc: 0.738 (best 0.738), test acc:

In [5]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [6]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.950, val acc: 0.156 (best 0.156), test acc: 0.144 (best 0.144)
In epoch 5, loss: 1.887, val acc: 0.364 (best 0.364), test acc: 0.351 (best 0.351)
In epoch 10, loss: 1.756, val acc: 0.636 (best 0.636), test acc: 0.653 (best 0.653)
In epoch 15, loss: 1.553, val acc: 0.680 (best 0.682), test acc: 0.713 (best 0.704)
In epoch 20, loss: 1.284, val acc: 0.702 (best 0.702), test acc: 0.736 (best 0.736)
In epoch 25, loss: 0.977, val acc: 0.746 (best 0.746), test acc: 0.764 (best 0.764)
In epoch 30, loss: 0.678, val acc: 0.758 (best 0.758), test acc: 0.776 (best 0.776)
In epoch 35, loss: 0.431, val acc: 0.754 (best 0.766), test acc: 0.780 (best 0.778)
In epoch 40, loss: 0.259, val acc: 0.762 (best 0.766), test acc: 0.776 (best 0.778)
In epoch 45, loss: 0.153, val acc: 0.764 (best 0.766), test acc: 0.780 (best 0.778)
In epoch 50, loss: 0.093, val acc: 0.768 (best 0.768), test acc: 0.783 (best 0.783)
In epoch 55, loss: 0.059, val acc: 0.770 (best 0.770), test acc: 0.783 (best 0

In [7]:
def u_mul_e_udf(edges):
    return {"m": edges.src["h"] * edges.data["w"]}

In [8]:
def mean_udf(nodes):
    return {"h_N": nodes.mailbox["m"].mean(1)}